<a href="https://colab.research.google.com/github/1361/Book-TDD-Web-Dev-Python/blob/master/notebooks/SBIR_pasture_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# Colab: Convert 4-band UInt16 (R,G,B,NIR) GeoTIFF -> App GeoTIFF
# Output bands:
#   1: R (uint8, stretched to 1..255 for VALID pixels)
#   2: G (uint8, stretched to 1..255 for VALID pixels)
#   3: B (uint8, stretched to 1..255 for VALID pixels)
#   4: NDVI (uint8, 0..255; 0 reserved for invalid)
#   5: Classification placeholder (uint8; 5 reserved for valid, 0 invalid)
#
# Key updates:
#   - DO NOT inherit nodata from source: explicitly remove it from output profile
#   - Write an internal dataset mask
#   - Reserve 0 ONLY for invalid pixels (prevents "dark pixels become nodata")
# ============================================================

!pip -q install rasterio

import numpy as np
import rasterio

in_tif  = "/content/sbir_22.tif"         # upload your raw tif here
out_tif = "/content/processed_tif.tif"   # output tif for your web app

def derive_valid_mask(src, R16, G16, B16, N16):
    """
    For your imagery, we want to avoid overly aggressive masking.
    Best fallback: treat nodata ONLY where ALL bands are 0.
    (Even if a dataset mask exists, it may be aggressive and hide shadows.)
    """
    is_nodata = (R16 == 0) & (G16 == 0) & (B16 == 0) & (N16 == 0)
    return ~is_nodata

def stretch_to_u8_reserved0(arr, valid_mask, p_low=2, p_high=98):
    """
    Percentile stretch UInt16 -> UInt8, but:
      - VALID pixels map to 1..255 (never 0)
      - INVALID pixels become 0
    This prevents valid dark pixels from being treated as NoData when nodata=0 exists anywhere.
    """
    a = arr.astype(np.float32)
    v = a[valid_mask]

    out = np.zeros(arr.shape, dtype=np.uint8)  # default 0 for invalid
    if v.size == 0:
        return out

    lo = np.percentile(v, p_low)
    hi = np.percentile(v, p_high)
    if hi <= lo:
        hi = lo + 1.0

    scaled = (a - lo) / (hi - lo)
    scaled = np.clip(scaled, 0, 1)

    # map [0,1] -> [1,255] for valid pixels
    out_valid = (scaled * 254 + 1).round().astype(np.uint8)

    out[valid_mask] = out_valid[valid_mask]
    return out

with rasterio.open(in_tif) as src:
    # Raw bands (UInt16)
    R16 = src.read(1)
    G16 = src.read(2)
    B16 = src.read(3)
    N16 = src.read(4)

    profile = src.profile.copy()

    # Valid-data mask: only all-bands-zero treated as nodata
    valid = derive_valid_mask(src, R16, G16, B16, N16)

    # Stretch RGB for display, reserving 0 for invalid pixels
    R8 = stretch_to_u8_reserved0(R16, valid_mask=valid, p_low=2, p_high=98)
    G8 = stretch_to_u8_reserved0(G16, valid_mask=valid, p_low=2, p_high=98)
    B8 = stretch_to_u8_reserved0(B16, valid_mask=valid, p_low=2, p_high=98)

    # NDVI from original radiometry (float [-1,1])
    Rf = R16.astype(np.float32)
    Nf = N16.astype(np.float32)
    denom = (Nf + Rf)

    ndvi = np.full(Rf.shape, np.nan, dtype=np.float32)
    ok = valid & (denom != 0)
    ndvi[ok] = (Nf[ok] - Rf[ok]) / denom[ok]

    # NDVI [-1,1] -> [0,1] -> uint8 [0,255]
    # 0 reserved for invalid pixels
    ndvi01 = (ndvi + 1.0) / 2.0
    ndvi01 = np.clip(ndvi01, 0.0, 1.0)

    ndvi_u8 = np.zeros(Rf.shape, dtype=np.uint8)
    ndvi_u8[ok] = (ndvi01[ok] * 255).round().astype(np.uint8)

    # Classification placeholder: 5 where valid, else 0
    cls_u8 = np.zeros(Rf.shape, dtype=np.uint8)
    cls_u8[valid] = 5

    # --- Build output profile WITHOUT inheriting nodata ---
    out_profile = profile.copy()
    out_profile.update(
        dtype="uint8",
        count=5,
        compress="deflate",
        predictor=2,
        tiled=True,
        blockxsize=256,
        blockysize=256
    )

    # ✅ Critical: remove nodata from the output profile if it exists
    # (your input likely has nodata=0; we do NOT want that carried forward)
    out_profile.pop("nodata", None)

    with rasterio.open(out_tif, "w", **out_profile) as dst:
        dst.write(R8, 1)
        dst.write(G8, 2)
        dst.write(B8, 3)
        dst.write(ndvi_u8, 4)
        dst.write(cls_u8, 5)

        # ✅ Write dataset mask (255 valid, 0 invalid)
        dst.write_mask((valid.astype(np.uint8)) * 255)

        # Extra safety: explicitly clear nodata on dataset handle (if supported)
        try:
            dst.nodata = None
        except Exception:
            pass

print("Wrote:", out_tif)

# Sanity checks
with rasterio.open(out_tif) as out:
    print("Output CRS:", out.crs)
    print("Output bands:", out.count)
    print("Output dtype:", out.dtypes)
    print("Output nodata (should be None):", out.nodata)
    print("Mask band present? (should be True):", out.mask_flag_enums)
    print("Band1 min/max (should be >=1 for valid):", out.read(1).min(), out.read(1).max())


Wrote: /content/processed_tif.tif
Output CRS: EPSG:32613
Output bands: 5
Output dtype: ('uint8', 'uint8', 'uint8', 'uint8', 'uint8')
Output nodata (should be None): None
Mask band present? (should be True): ([<MaskFlags.per_dataset: 2>], [<MaskFlags.per_dataset: 2>], [<MaskFlags.per_dataset: 2>], [<MaskFlags.per_dataset: 2>], [<MaskFlags.per_dataset: 2>])
Band1 min/max (should be >=1 for valid): 0 255
